In [1]:
import os
import requests
import hashlib

In [2]:
urniki = [
    "fri-2016_2017-zimski",
    "fri-2016_2017-zimski-drugi-teden",
    "fri-2016_2017-letni",
    "fri-2017_2018-zimski",
    "fri-2017_2018-letni-1-teden",
    "fri-2018_2019-zimski",
    "fri-2018_2019-letni-1-13",
    "fri-2018_2019-zimski-1-6"
]

Povezava do urnika je oblike:

    https://urnik.fri.uni-lj.si/timetable/[urnik]/allocations?student=[vpisna]

kjer `[urnik]` predstavlja enega od objavljenih urnikov, `[vpisna]` pa vpisno številko študenta.

Datoteka `vpisne.txt` vsebuje vpisne številke študentov, ki so izbrali predmet PR v študijskih letih 2016-2017, 2017-2018 in 2018-2019. Datoteka ni priložena.

In [3]:
vpisne = [x.strip() for x in open('vpisne.txt').readlines()]
len(vpisne)

158

Pripravi preslikavo vpisna -> anonimizirana_vpisna.

Prepričaj se, da je 8 zadnjih črt v preslikavi zgostitvene funkcije (hash) dovolj za bijektivno preslikavo za dane vpisne.

In [4]:
vpisna2hash = {}
for vpisna in vpisne:
    m = hashlib.sha256()
    m.update(vpisna.encode())
    vpisna_anon = str(m.hexdigest())[-8:]
    assert vpisna2hash.setdefault(vpisna, vpisna_anon) == vpisna_anon   

Za vsak `urnik` shrani urnike vseh `vpisnih`.

In [5]:
for urnik in urniki:
    if not os.path.isdir(urnik):
        os.makedirs(urnik)
    for vpisna in vpisne:
        url = f"https://urnik.fri.uni-lj.si/timetable/{urnik}/allocations?student={vpisna}"
        html = requests.get(url).text
        # vpisno zamenjaj z (anonimizirano) kodo
        vpisna_anon = vpisna2hash[vpisna]
        html = html.replace(vpisna, vpisna_anon)
        # in shrani
        with open(f'{urnik}/{vpisna_anon}.html', 'wt') as fout:
            fout.write(html)